In [1]:
# 本代码是探究指定时间进入指定地点的人群

In [4]:
# 导入模块

In [2]:
import pandas as pd
import os
from os import path

In [3]:
import warnings

warnings.filterwarnings(action='ignore')

In [10]:
from datetime import time

In [5]:
# 定义函数

In [8]:
def get_data(file_name):
    '''
    传入的是文件路径
    读取并对数据进行处理
    '''
    file_extension = path.splitext(file_name)[1]
    if file_extension == '.csv':
        data = pd.read_csv(file_name, encoding='gbk')
    elif file_extension == '.xlsx':
        data = pd.read_excel(file_name, encoding='gbk')
    drop_col = ['imei_', 'area_', 'msisdn_']
    data.drop(drop_col, axis=1, inplace=True)
    col = data.columns
    for each in col:
        if data[each].dtype == object:
            data[each] = data[each].str.replace('\t', '')
    print('data load successfully')
    return data

In [6]:
os.listdir('../Project Data/')

os.chdir('../Project Data/') # 切换到数据所在目录

In [9]:
data = get_data('20190524-001.csv')

data load successfully


In [11]:
data.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
0,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
1,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
3,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
4,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [13]:
data1 = data.copy()

In [14]:
data1.index = pd.to_datetime(data.update_time_)

In [15]:
data1.head()

,imsi_,update_time_,lac_,cellid_,status_,ap_type_,device_id_,eventid_
update_time_,,,,,,,,
2019-05-08 20:09:29,460110413044304,2019-05-08 20:09:29,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:09:41,460110413042577,2019-05-08 20:09:41,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:14,460110425023300,2019-05-08 20:10:14,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:10:40,460110714141865,2019-05-08 20:10:40,0,0,拒绝,FDD-LTE,DX-SZSC001,Normal Lau
2019-05-08 20:08:55,460016692088409,2019-05-08 20:08:55,0,0,拒绝,FDD-LTE,LT-SZSC001,Normal Lau


In [19]:
temp = data1.loc[time(8,0):time(10, 0)]
d = {}
for i,j in temp.groupby(temp.index.day).imsi_:
    d[i] = set(j.to_list())

In [21]:
# 开始定位，某个时间段出现什么人
d.keys() # 时间区间，表示9号，10号，11号

dict_keys([9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

In [32]:
res = None
day = ''
for i, j in d.items():
    if not res:
        day = ','.join([day, str(i)])
        res = j
    else:
        day = ','.join([day, str(i)])
        res &= j
print(f'用户：{res} 在这个月的 {day[1:]} 天的 8-10 点出现了')

用户：{460110267853698, 460000211720632, 460002406691111, 460029571752973, 460110412995758, 460014926623352, 460004202121725} 在这个月的 9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24 天的 8-10 点出现了


In [23]:
s = ''
','.join(['1', s])

'1,'